In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Function
import numpy as np
import torchvision
from BNN import *
from spikingjelly.activation_based import functional

: 

In [5]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='/home/yanggl/code', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='/home/yanggl/code', train=False, transform=transform, download=True)

In [2]:
#cifar10
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='/home/yanggl/code', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='/home/yanggl/code', train=False, transform=transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = BinaryConv2d(3, 6, kernel_size=3, stride=1, padding=0, bias=False)
        self.conv2 = BinaryConv2d(6, 12, kernel_size=3, stride=1, padding=0, bias=False)
        self.conv3 = BinaryConv2d(12, 24, kernel_size=3, stride=1, padding=0, bias=False)
        self.sn1 = BinaryActivation()
        self.sn2 = BinaryActivation()
        self.sn3 = BinaryActivation()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = BinaryLinear(24*6*6, 10, bias=False)
    def forward(self, x):
        #大于127的变成1，小于等于127的变成0
        x = torch.where(x>0.5, torch.tensor(1.), torch.tensor(-1.))
        x = self.conv1(x)
        #x = self.pool1(x)
        #print(x.int())
        #x = self.pool1(x)
        #print(x.int())
        # x = torch.where(x>0, torch.tensor(1.), torch.tensor(-1.))
        x = self.sn1(x)
        #print(x.int())
        x = self.conv2(x)
        #print(x.int())
        #x = self.pool2(x)
        #x = self.relu2(x)
        #print(x.int())
        x = self.pool2(x) 
        x = self.sn2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.sn3(x)
        #print(x.shape)
        # x = torch.where(x>0, torch.tensor(1.), torch.tensor(-1.))
        #x = self.conv3(x)
        #x = self.pool3(x)
        #x = torch.where(x>0, torch.tensor(1.), torch.tensor(-1.))
        #print(x.int())
        x = x.view(x.size(0), -1)
        #print(x.int())
        x = self.fc1(x)
        # x = self.relu3(x)
        # x = self.fc2(x)
        # x = self.relu3(x)
        # x = self.fc3(x)
        return x

In [4]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
net = LeNet()
from tqdm import tqdm
np.int = int
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
criterion = nn.CrossEntropyLoss()
LR = 0.001

optimizer = optim.Adam(net.parameters(), lr=LR)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
EPOCH = 10
device = torch.device('cuda:0')
net.to(device)
for epoch in range(EPOCH):
    net.train()
    for i ,data in enumerate(dataloader):
        inputs, labels = data
        inputs = inputs.to(device).cuda()
        labels = labels.to(device).cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        functional.reset_net(net)
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i,data in enumerate(testloader):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
torch.save(net.state_dict(), 'weight/lenet.pth')

/home/yanggl/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1,     1] loss: 2.426
[1,   101] loss: 2.051
[1,   201] loss: 1.642
[1,   301] loss: 1.808
[1,   401] loss: 1.812
[1,   501] loss: 1.905
[1,   601] loss: 1.934
[1,   701] loss: 1.902
[1,   801] loss: 2.138
[1,   901] loss: 2.388
[1,  1001] loss: 2.206
[1,  1101] loss: 1.820
[1,  1201] loss: 2.142
[1,  1301] loss: 2.430
[1,  1401] loss: 1.569
[1,  1501] loss: 1.575
[1,  1601] loss: 2.034
[1,  1701] loss: 1.620
[1,  1801] loss: 1.918
[1,  1901] loss: 2.428
[1,  2001] loss: 2.139
[1,  2101] loss: 1.948
[1,  2201] loss: 2.781
[1,  2301] loss: 1.509
[1,  2401] loss: 2.486
[1,  2501] loss: 2.000
[1,  2601] loss: 2.128
[1,  2701] loss: 2.675
[1,  2801] loss: 2.038
[1,  2901] loss: 2.122
[1,  3001] loss: 2.032
[1,  3101] loss: 1.844
Accuracy of the network on the 10000 test images: 29 %
[2,     1] loss: 1.893
[2,   101] loss: 1.956
[2,   201] loss: 1.886
[2,   301] loss: 1.768
[2,   401] loss: 2.197
[2,   501] loss: 2.073
[2,   601] loss: 1.836
[2,   701] loss: 1.553
[2,   801] loss: 2.026
[2

KeyboardInterrupt: 

In [6]:
class scale_Bconvd(BinaryConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False):
        super(scale_Bconvd, self).__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
    def forward(self, x):
        w = self.weight
        bw = BinaryWeight.apply(w)
        scaling_factor = torch.mean(torch.mean(torch.mean(torch.mean(abs(w),dim=3,keepdim=True),dim=2,keepdim=True),dim=1,keepdim=True),dim=0,keepdim=True)
        scaling_factor = scaling_factor.detach()
        #print(scaling_factor)
        # bw = scaling_factor * BinaryWeight.apply(w)
    
        return F.conv2d(x, bw, self.bias, self.stride,
                    self.padding, self.dilation, self.groups)
class scale_Blinear(BinaryLinear):
    def __init__(self, in_features, out_features, bias=False):
        super(scale_Blinear, self).__init__(in_features, out_features, bias)
        
    def forward(self, x):

        w = self.weight
        bw = BinaryWeight.apply(w)
        # print(w.shape)
        scaling_factor = torch.mean(torch.mean(abs(w),dim=1,keepdim=True),dim=0,keepdim=True)
        scaling_factor = scaling_factor.detach()
        #print(scaling_factor)
        # bw = scaling_factor * BinaryWeight.apply(w)
        
        return F.linear(x, bw, self.bias)
    
class scale_leNet(nn.Module):
    def __init__(self, num_classes=10, T=4):
        super().__init__()
        self.T = T
        self.conv1 = scale_Bconvd(1, 3, kernel_size=3, stride=1, padding=0, bias=False)
        self.sn1 = BinaryActivation()
        self.conv2 = scale_Bconvd(3, 6, kernel_size=3, stride=1, padding=0, bias=False)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.sn2 = BinaryActivation()
        self.fc1 = scale_Blinear(6*12*12, 10, bias=False)
        # self.relu3 = nn.ReLU()
        # self.fc2 = scale_Blinear(120, 84, bias=False)
        # self.relu4 = nn.ReLU()
        # self.fc3 = scale_Blinear(84, num_classes, bias=False)
    def forward(self, x):
        x = torch.where(x>0.5, torch.tensor(1.), torch.tensor(-1.))
        x = self.conv1(x)
        #print(x.int())
        #x = self.pool1(x)
        #print(x.int())
        x = self.sn1(x)
        #print(x.int())
        x = self.conv2(x)
        #print(x.int())
        #x = self.pool2(x)
        #x = self.relu2(x)
        #print(x.int())
        x = self.pool2(x)
        x = self.sn2(x)
        #x = self.conv3(x)
        #x = self.pool3(x)
        #x = torch.where(x>0, torch.tensor(1.), torch.tensor(-1.))
        #print(x.int())
        x = x.view(x.size(0), -1)
        #print(x.int())
        x = self.fc1(x)
        # x = self.relu3(x)
        # x = self.fc2(x)
        # x = self.relu3(x)
        # x = self.fc3(x)
        return x

In [7]:
#将权重加载，二值化，保存为另外一个pth文件
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)
net = LeNet()
device = torch.device('cuda:0')
net.to(device)
net.load_state_dict(torch.load('weight/lenet.pth',weights_only=True))
for name, param in net.named_parameters():
    if 'weight' in name:
        param.data = BinaryWeight.apply(param.data)
torch.save(net.state_dict(), 'weight/lenet_binary.pth')
#加载二值化后的权重
net.load_state_dict(torch.load('weight/lenet_binary.pth',weights_only=True))
net.to(device)
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(testloader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 91 %


In [8]:
#将网络第一层的权重导出
conv1_weight = net.conv1.weight.data
#print(conv1_weight)
conv1_weight = conv1_weight.view(-1)
#print(conv1_weight.shape)
conv1_weight_int = conv1_weight.int()
#print(conv1_weight_int)
#将第一层的权重导出为txt文件
with open('test_array/test_conv1_weight_txt.txt', 'w') as f:
    for weight in conv1_weight_int:
        weight = weight.item()
        if weight < 0:
            weight = 0
        f.write(format(weight, 'b') + '\n')
#将网络第二层的权重导出
conv2_weight = net.conv2.weight.data
#print(conv2_weight.shape)
#print(conv2_weight)
conv2_weight = conv2_weight.view(-1)
#print(conv2_weight.shape)
conv2_weight_int = conv2_weight.int()
#print(conv2_weight_int)
#将第二层的权重导出为txt文件
with open('test_array/test_conv2_weight_txt.txt', 'w') as f:
    for weight in conv2_weight_int:
        weight = weight.item()
        if weight < 0:
            weight = 0
        f.write(format(weight, 'b') + '\n')
#将网络第三层的权重导出
fc1_weight = net.fc1.weight.data
print(fc1_weight)
print(fc1_weight.shape)
fc1_weight = fc1_weight.view(-1)
print(fc1_weight.shape)
fc1_weight_int = fc1_weight.int()
print(fc1_weight_int)
#将第三层的权重导出为txt文件
with open('test_array/test_fc1_weight_txt.txt', 'w') as f:
    for weight in fc1_weight_int:
        weight = weight.item()
        if weight < 0:
            weight = 0
        f.write(format(weight, 'b') + '\n')
    

tensor([[ 1., -1.,  1.,  ..., -1., -1.,  1.],
        [ 1., -1.,  1.,  ...,  1.,  1.,  1.],
        [-1.,  1., -1.,  ...,  1.,  1., -1.],
        ...,
        [ 1., -1.,  1.,  ..., -1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1., -1.],
        [ 1., -1.,  1.,  ..., -1.,  1.,  1.]], device='cuda:0')
torch.Size([10, 864])
torch.Size([8640])
tensor([ 1, -1,  1,  ..., -1,  1,  1], device='cuda:0', dtype=torch.int32)


In [10]:
class watch_scale_leNet(nn.Module):
    def __init__(self, num_classes=10, T=4):
        super().__init__()
        self.T = T
        self.conv1 = scale_Bconvd(1, 3, kernel_size=3, stride=1, padding=0, bias=False)
        self.sn1 = BinaryActivation()   
        self.conv2 = scale_Bconvd(3, 6, kernel_size=3, stride=1, padding=0, bias=False)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.sn2 = BinaryActivation()
        self.fc1 = scale_Blinear(6*12*12, 10, bias=False)
    def forward(self, x):
        x = torch.where(x>127, torch.tensor(1.), torch.tensor(-1.))
        #print(x.shape)
        #print(x.int())
        x = self.conv1(x)
        x = self.sn1(x)
        conv1_out_int = torch.where(x==1., torch.tensor(1.), torch.tensor(0.)).int()
        with open('test_array/test_conv1_output_txt.txt', 'w') as f:
            for i in range(26):
                for j in range(26):
                    f.write(format(conv1_out_int[0][0][i][j].item(), 'b') + '\n')
        # print(x.shape)
        # print(x.int())
        x = self.conv2(x)
        print(x.shape)
        print(x.int())
        #print(self.conv2.weight)
        x = self.pool2(x)
        x = self.sn2(x)
        print(x.shape)
        # print(x.int())
        x = x.view(x.size(0), -1)
        print(x.shape)
        print(x.int())
        x_b = torch.where(x==1., torch.tensor(1.), torch.tensor(0.))
        x_b_int = x_b.int()
        with open('test_array/test_fc1_input_txt.txt', 'w') as f:
            for i in range(6*12*12):
                f.write(format(x_b_int[0][i].item(), 'b') + '\n')
        #print(self.fc1.weight[0])
        x = self.fc1(x)
        # x = self.relu3(x)
        # x = self.fc2(x)
        # x = self.relu4(x)
        # x = self.fc3(x)
        return x

net = watch_scale_leNet()
net.load_state_dict(torch.load('weight/lenet_binary.pth',weights_only=True))
image, label = train_dataset[0]
image_b = torch.where(image>0.5, torch.tensor(1.), torch.tensor(0.))
image_b_int = image_b.int()
with open('test_array/test_image_b_txt.txt', 'w') as f:
    for i in range(28):
        for j in range(28):
            f.write(format(image_b_int[0][i][j].item(), 'b') + '\n')
image = image*255
image_int = image.int()
with open('test_array/test_image_txt.txt', 'w') as f:
    for i in range(28):
        for j in range(28):
            f.write(format(image_int[0][i][j].item(), '08b') + '\n')
image = image
print(image.shape)
output = net(image.unsqueeze(0))
print(output.int())


torch.Size([1, 28, 28])
torch.Size([1, 6, 24, 24])
tensor([[[[-11, -11, -11,  ..., -11, -11, -11],
          [-11, -11, -11,  ..., -11, -11, -11],
          [-11, -11, -11,  ..., -11,  -9, -11],
          ...,
          [-13, -13,  -7,  ..., -11, -11, -11],
          [-15,  -9,  -1,  ..., -11, -11, -11],
          [-15,  -9,  -1,  ..., -11, -11, -11]],

         [[  7,   7,   7,  ...,   7,   7,   7],
          [  7,   7,   7,  ...,   7,   7,   7],
          [  7,   7,   7,  ...,   7,   9,   7],
          ...,
          [  5,   1,  -1,  ...,   7,   7,   7],
          [  3,   1,  -7,  ...,   7,   7,   7],
          [  3,   1,  -3,  ...,   7,   7,   7]],

         [[ -5,  -5,  -5,  ...,  -5,  -5,  -5],
          [ -5,  -5,  -5,  ...,  -5,  -5,  -5],
          [ -5,  -5,  -5,  ...,  -1,  -3,  -5],
          ...,
          [ -3,   1,   3,  ...,  -5,  -5,  -5],
          [ -1,   1,   9,  ...,  -5,  -5,  -5],
          [ -1,   1,  -3,  ...,  -5,  -5,  -5]],

         [[ 13,  13,  13,  ...,  1

In [11]:
import torch
import torch.nn as nn
weight_conv2 = torch.tensor([[-1.,-1.,-1.],[-1.,1.,-1.],[-1.,1.,-1.]])
test_conv2 = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=0, bias=False)
with torch.no_grad():
    test_conv2.weight.copy_(weight_conv2)
#input来自一个 txt 文件
input_data = []
with open('/home/ygl/code/BNN_accelerator/train/test_array/test_conv1_output_txt.txt', 'r') as f:
    for line in f:
        input_data.append(int(line.strip(), 2))
input_tensor = torch.tensor(input_data, dtype=torch.float32).view(1, 1, 26, 26)
input_tensor = torch.where(input_tensor==1., torch.tensor(1.), torch.tensor(-1.))
output = test_conv2(input_tensor)
print(output)




tensor([[[[ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
            5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
            5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
            5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  3.,
            5.,  3.,  3.,  3.,  3.,  3.,  5.,  3.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  3.,  5.,  3.,  3.,  3.,  3.,  1.,
            3.,  1.,  1.,  1.,  1.,  3.,  3.,  3.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  3.,  5.,  3.,  1.,  3.,  1.,  1.,  1.,  1., -1.,
           -3., -3., -5., -3., -3., -1., -1.,  3.,  5.,  5.],
          [ 5.,  5.,  5.,  5.,  1.,  5.,  1., -1., -3., -5., -5., -5., -5., -5.,
           -5., -1., -5., -1., -1.,  1.,  3.,  5.,  5.,  5.],